# 🏟️ Soccer Banner Segmentation - Inference Pipeline

Questo notebook esegue l'inferenza sui modelli addestrati nella pipeline di training,
applicando la segmentazione dei banner pubblicitari su video e immagini.

## Funzionalità Principali
1. **Selezione Modello**: Scelta facile tra le diverse versioni di modelli disponibili
2. **Chroma Key Video**: Applicazione di green screen sui banner rilevati
3. **Confronto Modelli**: Visualizzazione side-by-side Expert vs Specialist

---

## 📚 1. Import delle Librerie

In [ ]:
import os
import re
from pathlib import Path

import numpy as np
import cv2
import matplotlib.pyplot as plt

import torch
from ultralytics import YOLO  # type: ignore

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("✅ Ambiente di inferenza pronto.")
print(f"   ⚙️  Dispositivo: {device.upper()}")
print(f"   🔧 PyTorch: {torch.__version__}")
if torch.cuda.is_available():
    print(f"   🎮 GPU: {torch.cuda.get_device_name(0)}")

## 📁 2. Configurazione Percorsi

In [ ]:
BASE_DIR = Path(os.getcwd())
INPUT_DIR = BASE_DIR / "input"
OUTPUT_DIR = BASE_DIR / "output"
MODEL_DIR = BASE_DIR / "model"

VIDEO_OUT = OUTPUT_DIR / "FINAL_VIDEOS"
VIDEO_OUT.mkdir(parents=True, exist_ok=True)

VIDEO_IN = INPUT_DIR / 'video_input.mp4'

print("✅ Percorsi configurati:")
print(f"   📂 Input:        {INPUT_DIR}")
print(f"   📂 Output:       {OUTPUT_DIR}")
print(f"   📂 Model:        {MODEL_DIR}")

## 🔍 3. Funzioni per Selezione Modello

In [ ]:
def list_available_models(model_dir: Path, model_type: str | None = None) -> list:
    """Elenca tutti i modelli disponibili."""
    models = []
    if not model_dir.exists():
        return models
    
    for item in sorted(model_dir.iterdir()):
        if item.is_dir():
            weights_path = item / 'best.pt'
            if weights_path.exists():
                if model_type is None or item.name.startswith(model_type):
                    models.append((item.name, weights_path))
    return models


def get_latest_model(model_dir: Path, model_type: str) -> Path | None:
    """Ottiene l'ultima versione di un tipo di modello."""
    models = list_available_models(model_dir, model_type)
    if not models:
        return None
    
    def extract_version(name):
        match = re.search(r'_v(\d+)$', name)
        return int(match.group(1)) if match else 0
    
    models.sort(key=lambda x: extract_version(x[0]), reverse=True)
    return models[0][1]


def display_available_models(model_dir: Path):
    """Mostra tutti i modelli disponibili."""
    print("\n" + "="*60)
    print("📦 MODELLI DISPONIBILI")
    print("="*60)
    
    expert_models = list_available_models(model_dir, 'expert_model')
    specialist_models = list_available_models(model_dir, 'specialist_model')
    
    print("\n🏋️ Expert Models:")
    for name, path in expert_models:
        print(f"   • {name}")
    if not expert_models:
        print("   (nessuno)")
    
    print("\n🎯 Specialist Models:")
    for name, path in specialist_models:
        print(f"   • {name}")
    if not specialist_models:
        print("   (nessuno)")
    
    print("\n" + "="*60)


display_available_models(MODEL_DIR)

## ⚙️ 4. Selezione Versione Modello

**Modifica le variabili per scegliere la versione:**

In [ ]:
# ============================================================================
# SELEZIONE MODELLO
# None = ultima versione, oppure 'v1', 'v2', etc.
# ============================================================================

EXPERT_VERSION = None
SPECIALIST_VERSION = None

# ============================================================================

# Risolvi i path
if EXPERT_VERSION is None:
    EXPERT_WEIGHTS = get_latest_model(MODEL_DIR, 'expert_model')
else:
    EXPERT_WEIGHTS = MODEL_DIR / f'expert_model_{EXPERT_VERSION}' / 'best.pt'
    if not EXPERT_WEIGHTS.exists():
        EXPERT_WEIGHTS = None

if SPECIALIST_VERSION is None:
    SPECIALIST_WEIGHTS = get_latest_model(MODEL_DIR, 'specialist_model')
else:
    SPECIALIST_WEIGHTS = MODEL_DIR / f'specialist_model_{SPECIALIST_VERSION}' / 'best.pt'
    if not SPECIALIST_WEIGHTS.exists():
        SPECIALIST_WEIGHTS = None

# Fallback a output/
if EXPERT_WEIGHTS is None:
    fallback = OUTPUT_DIR / 'expert_model' / 'weights' / 'best.pt'
    if fallback.exists():
        EXPERT_WEIGHTS = fallback

if SPECIALIST_WEIGHTS is None:
    fallback = OUTPUT_DIR / 'specialist_model' / 'weights' / 'best.pt'
    if fallback.exists():
        SPECIALIST_WEIGHTS = fallback

# Estrai nomi versione per uso successivo
EXPERT_VERSION_NAME = EXPERT_WEIGHTS.parent.name if EXPERT_WEIGHTS else 'N/A'
SPECIALIST_VERSION_NAME = SPECIALIST_WEIGHTS.parent.name if SPECIALIST_WEIGHTS else 'N/A'

print("\n📦 Modelli selezionati:")
print(f"   {'✅' if EXPERT_WEIGHTS else '❌'} Expert:     {EXPERT_VERSION_NAME}")
print(f"   {'✅' if SPECIALIST_WEIGHTS else '❌'} Specialist: {SPECIALIST_VERSION_NAME}")
print(f"\n🎬 Video Input: {'✅' if VIDEO_IN.exists() else '❌'} {VIDEO_IN.name}")

## 🎬 5. Funzione Chroma Key

In [ ]:
def apply_chroma_key(
    model_path: Path,
    video_path: Path,
    output_name: str,
    conf_threshold: float = 0.10,  # Ridotto standard a 0.10
    multi_pass: bool = True
) -> Path | None:
    """Applica chroma key sui banner rilevati."""
    
    if model_path is None or not os.path.exists(model_path):
        print(f"❌ Modello non trovato: {model_path}")
        return None
    
    if not os.path.exists(video_path):
        print(f"❌ Video non trovato: {video_path}")
        return None
    
    print(f"📦 Caricamento modello: {model_path.parent.name}")
    model = YOLO(str(model_path)).to(device)
    
    cap = cv2.VideoCapture(str(video_path))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"🎬 Video: {width}x{height} @ {fps:.1f}fps ({total_frames} frames)")
    
    out_file = VIDEO_OUT / output_name
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # type: ignore
    writer = cv2.VideoWriter(str(out_file), fourcc, fps, (width, height))
    
    # KERNEL TUNING: 
    # 1. Close molto ampio (21x21) per riempire i buchi grandi
    # 2. Dilate minimo (3x3 iter=1) per evitare di coprire la rete
    kernel_dilate = np.ones((3, 3), np.uint8)
    kernel_close = np.ones((21, 21), np.uint8)
    
    print(f"⏳ Elaborazione: {output_name}")
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if multi_pass:
            # Confidenza più bassa per trovare parti mancanti
            results_high = model.predict(frame, conf=conf_threshold * 0.8, imgsz=1280, device=device, verbose=False, retina_masks=True)
            results_std = model.predict(frame, conf=conf_threshold * 0.6, imgsz=1024, device=device, verbose=False, retina_masks=True)
            
            masks_combined = []
            if results_high[0].masks is not None:
                masks_combined.append(results_high[0].masks.data)  # type: ignore
            if results_std[0].masks is not None:
                masks_combined.append(results_std[0].masks.data)  # type: ignore
            
            if masks_combined:
                all_masks = torch.cat(masks_combined, dim=0)
                combined = torch.any(all_masks, dim=0).int() * 255  # type: ignore
            else:
                combined = torch.zeros((height, width), dtype=torch.int32)
        else:
            results = model.predict(frame, conf=conf_threshold, imgsz=1280, device=device, verbose=False, retina_masks=True)
            if results[0].masks is not None:
                combined = torch.any(results[0].masks.data, dim=0).int() * 255  # type: ignore
            else:
                combined = torch.zeros((height, width), dtype=torch.int32)
        
        mask_np = combined.cpu().numpy().astype(np.uint8)  # type: ignore
        mask_final = cv2.resize(mask_np, (width, height), interpolation=cv2.INTER_CUBIC)
        
        # POST-PROCESSING
        # 1. Chiudere i buchi (aggressive)
        mask_final = cv2.morphologyEx(mask_final, cv2.MORPH_CLOSE, kernel_close)
        # 2. Dilatazione minima
        mask_final = cv2.dilate(mask_final, kernel_dilate, iterations=1)
        # 3. Blur
        mask_final = cv2.GaussianBlur(mask_final, (5, 5), 0)
        
        frame[mask_final > 100] = [0, 255, 0]
        writer.write(frame)
        frame_count += 1
        
        if frame_count % 50 == 0:
            print(f"   📊 {frame_count}/{total_frames} ({frame_count/total_frames*100:.1f}%)")
    
    cap.release()
    writer.release()
    
    print(f"✅ Completato: {out_file}")
    print(f"   📁 Size: {out_file.stat().st_size / (1024*1024):.1f} MB")
    return out_file

## 🏋️ 6. Inferenza con Modello Expert

In [ ]:
if EXPERT_WEIGHTS is not None:
    print("="*60)
    print(f"🏋️ INFERENZA CON MODELLO EXPERT ({EXPERT_VERSION_NAME})")
    print("="*60)
    
    expert_output = apply_chroma_key(
        model_path=EXPERT_WEIGHTS,
        video_path=VIDEO_IN,
        output_name=f'video_{EXPERT_VERSION_NAME}.mp4'
    )
else:
    print("⚠️  Modello Expert non trovato.")

## 🎯 7. Inferenza con Modello Specialist

In [ ]:
if SPECIALIST_WEIGHTS is not None:
    print("="*60)
    print(f"🎯 INFERENZA CON MODELLO SPECIALIST ({SPECIALIST_VERSION_NAME})")
    print("="*60)
    
    specialist_output = apply_chroma_key(
        model_path=SPECIALIST_WEIGHTS,
        video_path=VIDEO_IN,
        output_name=f'video_{SPECIALIST_VERSION_NAME}.mp4'
    )
else:
    print("⚠️  Modello Specialist non trovato.")

## 📊 8. Confronto Visivo dei Modelli

In [ ]:
def compare_models_on_frames(video_path, expert_weights, specialist_weights, expert_name, specialist_name, num_frames=4):
    """Confronta Expert vs Specialist su frame estratti."""
    
    if expert_weights is None or specialist_weights is None:
        print("❌ Entrambi i modelli sono necessari per il confronto.")
        return
    
    if not video_path.exists():
        print(f"❌ Video non trovato: {video_path}")
        return
    
    print("="*60)
    print(f"📊 CONFRONTO: {expert_name} vs {specialist_name}")
    print("="*60)
    
    cap = cv2.VideoCapture(str(video_path))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = [int(i * total_frames / (num_frames + 1)) for i in range(1, num_frames + 1)]
    
    print("\n📦 Caricamento modelli...")
    expert_model = YOLO(str(expert_weights)).to(device)
    specialist_model = YOLO(str(specialist_weights)).to(device)
    print(f"   Expert: {expert_name}")
    print(f"   Specialist: {specialist_name}")
    
    fig, axes = plt.subplots(num_frames, 3, figsize=(15, 4*num_frames))
    if num_frames == 1:
        axes = [axes]
    
    for idx, frame_idx in enumerate(frame_indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            continue
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Expert
        expert_result = expert_model.predict(frame, conf=0.15, device=device, verbose=False)
        expert_frame = frame.copy()
        if expert_result[0].masks is not None:
            mask = expert_result[0].masks.data.cpu().numpy()  # type: ignore
            combined_mask = np.any(mask, axis=0).astype(np.uint8) * 255
            combined_mask = cv2.resize(combined_mask, (frame.shape[1], frame.shape[0]))
            expert_frame[combined_mask > 100] = [0, 255, 0]
        
        # Specialist
        specialist_result = specialist_model.predict(frame, conf=0.15, device=device, verbose=False)
        specialist_frame = frame.copy()
        if specialist_result[0].masks is not None:
            mask = specialist_result[0].masks.data.cpu().numpy()  # type: ignore
            combined_mask = np.any(mask, axis=0).astype(np.uint8) * 255
            combined_mask = cv2.resize(combined_mask, (frame.shape[1], frame.shape[0]))
            specialist_frame[combined_mask > 100] = [0, 255, 0]
        
        # Plot con versione nei titoli
        axes[idx][0].imshow(frame_rgb)
        axes[idx][0].set_title(f'Originale (Frame {frame_idx})')
        axes[idx][0].axis('off')
        
        axes[idx][1].imshow(cv2.cvtColor(expert_frame, cv2.COLOR_BGR2RGB))
        axes[idx][1].set_title(f'Expert ({expert_name})')
        axes[idx][1].axis('off')
        
        axes[idx][2].imshow(cv2.cvtColor(specialist_frame, cv2.COLOR_BGR2RGB))
        axes[idx][2].set_title(f'Specialist ({specialist_name})')
        axes[idx][2].axis('off')
    
    cap.release()
    plt.tight_layout()
    # Salvataggio confronto
    confronti_dir = OUTPUT_DIR / 'confronti'
    confronti_dir.mkdir(parents=True, exist_ok=True)
    base_name = 'confronto'
    file_name = f'{base_name}.png'
    counter = 1
    while (confronti_dir / file_name).exists():
        file_name = f'{base_name}_{counter}.png'
        counter += 1
    save_path = confronti_dir / file_name
    plt.savefig(save_path)
    print(f'\n   💾 Immagine salvata in: {save_path}')
    plt.show()
    print("\n✅ Confronto completato.")

# Esegui confronto
if EXPERT_WEIGHTS and SPECIALIST_WEIGHTS and VIDEO_IN.exists():
    compare_models_on_frames(
        VIDEO_IN, 
        EXPERT_WEIGHTS, 
        SPECIALIST_WEIGHTS,
        EXPERT_VERSION_NAME,
        SPECIALIST_VERSION_NAME,
        num_frames=4
    )